## Preparing downloaded binary SSMI & AMSR data for use in trajectory package
#### Developed by Johannes Mohrmann @UW
#### Modified by Ehsan Erfani @UW, 2022:
- Changes in parameters and functions to facilitate automation and to generalize the code for projects other than CSET   

In [1]:
### Change these parameters based on your own data path and availability
ssmi_basefolder = '/home/disk/eos3/erfani/Data/NEP/ssmi'
amsr_basefolder = '/home/disk/eos3/erfani/Data/NEP/amsr/'
YYYY = [2018, 2019, 2020, 2021]
MM   = ['06', '07', '08', '09']
sat_names = ['f16', 'f17', 'f18'] # Only for ssmi
data_vers = ['v07', 'v07', 'v08'] # Only for ssmi

In [2]:
########################
#### Libraries
from ftplib import FTP
import sys
import os
import glob
import numpy as np
import re
import xarray as xr
import datetime as dt


########################
###### Various functions

def convert_files_to_netcdf(files, dates, data_reader):
        datasets = [data_reader(i) for i in files]
        try:
            assert [np.allclose(i.variables['latitude'], datasets[0].variables['latitude']) for i in datasets]
            assert [np.allclose(i.variables['longitude'], datasets[0].variables['longitude']) for i in datasets]
        except AssertionError as e:
            print('caught')
            print(len(files))
            for i in datasets:
                print(i.variables['latitude'])
            raise e
        ds = xr.Dataset()
        ds['time'] = dates
        ds['latitude'] = datasets[0].variables['latitude']
        ds['longitude'] = datasets[0].variables['longitude']
        ds['orbit_segment'] = np.arange(2)
        for k, v in datasets[0].variables.items():
            if len(v.shape) == 3:
                concat_var = np.array([i.variables[k] for i in datasets])
                concat_var[concat_var<v.valid_min] = np.nan
                concat_var[concat_var>v.valid_max] = np.nan
                name = k if not k=='time' else 'UTCtime'
                ds[name] = (('time', 'orbit_segment', 'latitude', 'longitude'), concat_var)
            attrs = dict(long_name = v.long_name, units=v.units, valid_min=v.valid_min, valid_max=v.valid_max)
            if type(v.valid_min) == bool:
                attrs['valid_min'] = int(attrs['valid_min'])
                attrs['valid_max'] = int(attrs['valid_max'])
            ds[name].attrs = attrs
        ds.attrs['creation date'] = str(dt.datetime.utcnow())
        return ds    
    
def make_amsr_data_reader():
    from amsr_proc.amsr2_daily import AMSR2daily
    def read_data(filename):
        dataset = AMSR2daily(filename, missing=np.nan)
        if not dataset.variables: 
            print(filename)
            sys.exit('file not found')
        return dataset
    return read_data

def make_ssmi_data_reader():
    from ssmi_proc.ssmi_daily_v7 import SSMIdaily
    def read_data(filename):
        dataset = SSMIdaily(filename, missing=np.nan)
        if not dataset.variables: 
            print(filename)
            sys.exit('file not found')
        return dataset
    return read_data
    
def convert_amsr_to_netcdf(basefolder, YYYY, MM):
    data_reader = make_amsr_data_reader()
    for year in YYYY:
        for month in MM:
            save_name = os.path.join(basefolder, 'all', f'amsr_unified_{year}-{month}.nc')
            files = sorted(glob.glob(os.path.join(basefolder, f'y{year}', f'm{month}', f'f34_{year}{month}[0-9][0-9]v8.gz')))           
            dates = [dt.datetime.strptime(os.path.basename(i)[4:12], '%Y%m%d') for i in files]
            ds = convert_files_to_netcdf(files, dates, data_reader)
            ds.attrs['comments'] = "netcdf created by Ehsan Erfani @uw, original code developed by Johannes Mohrmann @uw,"+\
            " adapted from bytemaps from Remote Sensing Systems. http://remss.com/missions/amsr/"
            comp = dict(zlib=True, complevel=2)
            ds.to_netcdf(save_name, engine='h5netcdf', encoding={var: comp for var in ds.data_vars})

def convert_ssmi_to_netcdf(basefolder, YYYY, MM, sat_names, data_vers):
    data_reader = make_ssmi_data_reader()
    for year in YYYY:
        for month in MM:
            for sat,ver in zip(sat_names, data_vers):
                save_name = os.path.join(basefolder, 'all', f'ssmi_unified_{sat}_{year}-{month}.nc')
                print(sat)
                print(os.path.join(basefolder, sat))
                files = sorted(glob.glob(os.path.join(basefolder, sat, '*', f'y{year}', f'm{month}',\
                                                      f'[fF][0-9][0-9]_{year}{month}[0-9][0-9]v[0-9].gz')))
                dates = [dt.datetime.strptime(os.path.basename(i)[4:12], '%Y%m%d') for i in files]
                ds = convert_files_to_netcdf(files, dates, data_reader)
                ds.attrs['comments'] = "netcdf created by Ehsan Erfani @uw, original code developed by Johannes Mohrmann @uw,"+\
                " adapted from bytemaps from Remote Sensing Systems. http://remss.com/missions/ssmi/"
                comp = dict(zlib=True, complevel=2)
                ds.to_netcdf(save_name, engine='h5netcdf', encoding={var: comp for var in ds.data_vars})            

## Create NetCDF files

In [3]:
#######################################
### Run the main function to unify ssmi

if __name__ == "__main__":
    convert_ssmi_to_netcdf(ssmi_basefolder, YYYY, MM, sat_names, data_vers)

f16
/home/disk/eos3/erfani/Data/NEP/ssmi/f16
f17
/home/disk/eos3/erfani/Data/NEP/ssmi/f17
f18
/home/disk/eos3/erfani/Data/NEP/ssmi/f18
f16
/home/disk/eos3/erfani/Data/NEP/ssmi/f16
f17
/home/disk/eos3/erfani/Data/NEP/ssmi/f17
f18
/home/disk/eos3/erfani/Data/NEP/ssmi/f18
f16
/home/disk/eos3/erfani/Data/NEP/ssmi/f16
f17
/home/disk/eos3/erfani/Data/NEP/ssmi/f17
f18
/home/disk/eos3/erfani/Data/NEP/ssmi/f18
f16
/home/disk/eos3/erfani/Data/NEP/ssmi/f16
f17
/home/disk/eos3/erfani/Data/NEP/ssmi/f17
f18
/home/disk/eos3/erfani/Data/NEP/ssmi/f18


In [4]:
#######################################
### Run the main function to unify amsr

if __name__ == "__main__":
    convert_amsr_to_netcdf(amsr_basefolder, YYYY, MM)

### Legacy functions

In [ ]:
def get_ftp_files():
    server_url = 'ftp.remss.com'
    ftp = FTP(server_url)
    ftp.login(user=r'jkcm@uw.edu')#, passwd=r'jkcm@uw.edu')
    for year in ['2014', '2016']:
        ftp.cwd(f'/ascat/metopa/bmaps_v02.1/y{year}/')
        local_basedir = f'/home/disk/eos9/jkcm/Data/ascat/rss/{year}'
        for i in ftp.nlst():
            ftp.cwd(f'/ascat/metopa/bmaps_v02.1/y{year}/')
            ftp.cwd(i)
            if not os.path.exists(os.path.join(local_basedir, i)):
                os.makedirs(os.path.join(local_basedir, i))
            filenames = ftp.nlst()
            for filename in filenames:
                local_filename = os.path.join(local_basedir, i, filename)
                file = open(local_filename, 'wb')
                ftp.retrbinary('RETR '+ filename, file.write)
    ftp.close()
    
def convert_ascat_to_netcdf():
    data_reader = make_ascat_data_reader()
    basefolder = '/home/disk/eos9/jkcm/Data/ascat/rss/'
    for year in ['2014', '2015', '2016']:
        months = [f'm{i+1:02}' for i in range(12)]
        for month in months:
            save_name = f'/home/disk/eos9/jkcm/Data/ascat/rss/all/ascat_unified_{year}-{month[1:]}.nc'
            mdir = os.path.join(basefolder, year, month)
            files = [os.path.join(mdir, f) for f in os.listdir(mdir) if re.match(r'ascat_[0-9]{8}_v02.1.gz', f)]
            dates = [dt.datetime.strptime(os.path.basename(i)[6:14], '%Y%m%d') for i in files]
            ds = convert_files_to_netcdf(files, dates, data+reader)
            ds.attrs['comments'] = "netcdf created by jkcm@uw.edu, adapted from bytemaps from Remote Sensing Systems. " +\
                                "http://remss.com/missions/ascat/"
            comp = dict(zlib=True, complevel=2)
            ds.to_netcdf(save_name, engine='h5netcdf', encoding={var: comp for var in ds.data_vars})

            
def make_ascat_data_reader():
    from ascat_daily import ASCATDaily
    def read_data(filename):
        dataset = ASCATDaily(filename, missing=np.nan)
        if not dataset.variables: sys.exit('file not found')
        return dataset
    return read_data
